## 시계열 트랜스포머 

시계열 트랜트포머 모델은 시계열 예측을 위한 기본적인 인코더-디코더 구조의 트랜스포머다.

### 2개의 클래스

* TimeSeriesTransformerModel: 상당에 헤드가 없는 기본적인 트랜스포머
* TimeSeriesTransformerForPrediction: 상단에 분포 호데를 추가하여 시계열 예측에 사용할 수 있다. (확률적 예측 모델) 값을 직접 출력하지는 않는다.

### TimeSeriesTransformerForPrediction의 2가지 블록

인코더는`context_length`의 시계열 값을 입력(`past_values`)으로 받고, 디코더는 미래의 `prediction_length` 만큼의 시계열 값을 예측한다(`future_values`). 학습에는 `past_values`와 `future_values`를 제공해야한다.

### 가공하지 않은 past_values와 future_values 이외의 추가 특징

* `past_time_features`: 모델이 `future_values`에 추가할 시간적 특성. 트랜스포머 디코더의 "위치 인코딩" 역할을 한다.
* `future_time_features`: 모델이 `future_values`에 추가할 시간적 특성. 트랜스포머 디코더의 "위치 인코딩" 역할을 한다.
* `static_categorical_features`: 시간에 따라 변하지 않는 범주형 특성 모든 `past_values`, `future_values`에 동일한 값을 가진다. (ID, 카테고리 등)
* `static_real_features`: 시간에 따라 변하지 않는 실수값 특성 모든 `past_values`, `future_values`에 동일한 값을 가진다. (온도, 고도 등) 

### 모델의 훈련
시계열 트랜스포머 모델은 기계 번역을 위한 트랜스포머 훈련과 유사하게 "교사 강제(teacher-forcing)" 방식으로 훈련된다. 즉 훈련중에 `future_values`를 디코더의 입력으로 오른쪽으로 한 위치 이동시키고, `past_values`의 마지막 값을 앞에 붙인다. 각 시간 단계에서 모델은 다음 타겟을 예측해야 한다. 따라서 훈련 설정은 언어를 위한 GPT 모델과 유사하지만, `decoder_start_token_id` 개념이 없다. 

### 모델의 추론

추론 시에는 `past_values`의 최종값을 디코더의 입력으로 제공한다. 그 다음, 모델에서 샘플링하여 다음 시간 단계에서의 예측을 만들고, 이를 디코더에 공급하여 다음 예측을 만든다.


### TimeSeriesTransformerConfig class

#### Parameters

* `prediction_length`: (int), **예측길이**모델이 얼마나 멀리 미래를 예측할 지를 정의한다.
* `context_length`: (int, optional, defaults to `prediction_length`),**컨텍스트 길이** 모델에게 더 많은 과거 정보를 제공하여 더 나은 예측을 할 수 있도록 도와준다. 데이터 세트의 특성에 따라 이를 조정하는 것이 성능 향상에 도움이 될 수 있다.
* `distribution_output`: (string, optional, defaults "student_t"), **분포 출력** 올바른 분포를 선택하는 것("student_t", "normal", "negative_binomial")은 모델이 예측의 불확실성과 꼬리를 추정하는 방식에 영향을 준다
* `loss`: (string, optional, defaults to :"nll"), **손실함수** 현재는 음의 로그 우도("nagative log liklihood")만 지원한다.
* `input_size`: (int, optional, defaults to 1), **입력 크기** 단항 예측(univariate)에서 중요하다. 모델의 고려하는 타깃 변수의 수에 직접 영향을 미친다.
* `scaling`: (string, or bool, optional, defaults to "mean"), **스케일링** 입력 특징을 적절히 스케일링(mean, standard deviation)하는 것은 모델 수렴의 및 성능에 상당한 영향을 준다.
* `lags_sequence`: (list[int], optional, defaults to [1, 2, 3, 4, 5, 6, 7]), **지연 시퀀스** 시계열의 빈도에 따라 자연 특징을 맞춤 설정하면 시간적 의존성을 효과적으로 포착하는데 도움이 된다.
* `num_time_features`: (int, optional, defaults to 0), **시간 특징 수** 시간 특징을 추가하면 모델이 시간적 특성을 더 잘 이해하고 예측할 수 있다.
* `num_dynamic_real_features`: (int, optional, defaults to 0), **동적 실수 특징 수** 시간에 따라 변하는 실수값 특성
* `num_static_real_features`: (int, optional, defaults to 0), **정적 실수 특징 수** 시간에 따라 변하지 않는 실수값 특성
* `num_static_real_features`: (int, optional, defaults to 0), **정적 범주형 특징 수** 시간에 따라 변하지 않는 범주형 특성
* `cardinality`: (list[int], optional) **카디널리티** 각 정적 범주형 특성에 대한 카디널리티 `num_static_categorical_features`와 같은 길이의 정수 목록이어야 한며, `num_static_categorical_features`가 0보다 큰 경우 None일 수 없다.
* `embedding_dimension`: (list[int], optional), 각 정적 범주형 특성에 다한 임베딩차원 `num_static_categorical_features`와 같은 길이의 정수 목록이여야하며, `num_static_categorical_features`가 0보다 큰 경우 None일 수 없다.
* `d_model`: (int, optional, defaults to 64) 변형기 레이어의 차원
* `encoder_layers`: (int, optional, defaults to 2) 인코더 레이어의 수
* `decoder_layers`: (int, optional, defaults to 2) 디코더 레이어의 수
* `encoder_attention_heads`: (int, optional, defaults to 2) 변형 인코더의 각 주의 레이어에 대한 주의 헤드 수
* `decoder_attention_heads`: (int, optional, defaults to 2) 변형 디코더의 각 주의 레이어에 대한 주의 헤드 수
* `encoder_ffn_dim`: (int, optional, defaults to 32) 인코더의 "중간 (feed forward)" 차원
* `decoder_ffn_dim`: (int, optional, defaults to 32) 디코더의 "중간 (feed forward)" 차원
* `activation_function`: (str, function, optional, defaults to "gelu") 인코더와 디코더의 활성화 함수
* `dropout`: (float, optional, defaults to 0.1) 인코더와 디코더의 드롭아웃
* `encoder_layerdrop`: (float, optional, defaults to 0.1) 인코더 레이어 드롭
* `decoder_layerdrop`: (float, optional, defaults to 0.1) 디코더 레이어 드롭
* `attention_dropout`: (float, optional, defaults to 0.1) 어텐션 드롭아웃
* `activation_dropout`: (float, optional, defaults to 0.1) 활성화 드롭아웃
* `num_parallel_samples`: (int, optional, defaults to 100) 추론 시에 생성할 샘플 수
* `init_std`: (float, optional, defaults to 0.02) 초기화 표준 편차
* `use_cache`: (bool, optional, defaults to False) 캐시 사용 여부

In [1]:
# TImeSeriesTransformerConfig Sample

from transformers import TimeSeriesTransformerConfig, TimeSeriesTransformerModel

# Initializing a Time Series Transformer configuration with 12 time steps for prediction

config = TimeSeriesTransformerConfig(prediction_length=12)

model = TimeSeriesTransformerModel(config)

model.config

TimeSeriesTransformerConfig {
  "_attn_implementation_autoset": true,
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "attention_dropout": 0.1,
  "cardinality": [
    0
  ],
  "context_length": 12,
  "d_model": 64,
  "decoder_attention_heads": 2,
  "decoder_ffn_dim": 32,
  "decoder_layerdrop": 0.1,
  "decoder_layers": 2,
  "distribution_output": "student_t",
  "dropout": 0.1,
  "embedding_dimension": [
    0
  ],
  "encoder_attention_heads": 2,
  "encoder_ffn_dim": 32,
  "encoder_layerdrop": 0.1,
  "encoder_layers": 2,
  "feature_size": 9,
  "init_std": 0.02,
  "input_size": 1,
  "is_encoder_decoder": true,
  "lags_sequence": [
    1,
    2,
    3,
    4,
    5,
    6,
    7
  ],
  "loss": "nll",
  "model_type": "time_series_transformer",
  "num_dynamic_real_features": 0,
  "num_parallel_samples": 100,
  "num_static_categorical_features": 0,
  "num_static_real_features": 0,
  "num_time_features": 0,
  "prediction_length": 12,
  "scaling": "mean",
  "transformers_version

### TimeSeriesTransformerModel class

* config: TimeSeriesTransformerConfig 

분류 헤드가 없는 기본적인 시계열 트랜스포머 모델. 이 모델은 PreTrained에서 상속된다. 

#### Parameters

* `past_values` (`torch.FloatTensor` of shape (`batch_size`, `sequence_length`) or (`batch_size`, `sequence_length`, `input_size`)) — 미래를 예측하기 위해 컨텍스트 역할을 하는 시계열의 과거 값입니다. 이 텐서의 시퀀스 크기는 모델의 `context_length`보다 커야 하며, 모델은 더 큰 크기를 사용하여 지연 특징(추가 컨텍스트 역할을 하는 과거의 추가 값)을 구성합니다.
  
`sequence_length`는 `config.context_length` + `max(config.lags_sequence)`와 같으며, lags_sequence가 설정되지 않은 경우 기본적으로 `config.context_length` + 7과 같습니다(`config.lags_sequence`에서 가장 큰 look-back 인덱스는 7입니다). 속성 `_past_length`는 과거의 실제 길이를 반환합니다.

`past_values`는 트랜스포머 인코더가 입력으로 받는 값입니다 (선택적으로 추가 특징, 예를 들어 `static_categorical_features`, `static_real_features`, `past_time_features` 및 lags 포함). 선택적으로, 결측값은 0으로 대체하고 `past_observed_mask`를 통해 표시해야 합니다.

다변량 시계열의 경우, `input_size` > 1 차원이 필요하며 이는 시간 단계별 시계열의 변량 수에 해당합니다.

* `past_time_features` (`torch.FloatTensor` of shape (`batch_size`, `sequence_length`, `num_features`)) — 모델이 `past_values`에 내부적으로 추가할 필수 시간 특징입니다. 이러한 특징은 "월", "일" 등 벡터로 인코딩된 것일 수 있습니다(for instance as Fourier features). 또한, 시계열이 "어느 시점에 있는지"를 모델이 알 수 있도록 돕는 "age" 특징일 수도 있습니다. age 특징은 먼 과거 시간 단계에서는 작은 값을 가지며 현재 시간 단계에 가까워질수록 단조롭게 증가합니다. 휴일 특징도 시간 특징의 좋은 예입니다.
이러한 특징은 입력의 "위치 인코딩" 역할을 합니다. 따라서 BERT와 같은 모델에서는 위치 인코딩이 모델의 내부 매개변수로부터 처음부터 학습되는 반면, 시계열 트랜스포머는 추가 시간 특징을 제공해야 합니다. 시계열 트랜스포머는 `static_categorical_features`에 대한 추가 임베딩만 학습합니다.

Additional dynamic real covariances can be concatenated to this tensor, with the caveat that these features must but known at prediction time.

`num_features`는 `config.num_time_features` + `config.num_dynamic_real_features`와 같습니다.

* `past_observed_mask` (`torch.BoolTensor` of shape (`batch_size`, `sequence_length`) or (`batch_size`, `sequence_length`, `input_size`), optional) — 과거 값이 관찰되었는지 또는 누락되었는지를 나타내는 부울 마스크입니다. 마스크 값은 [0, 1]에서 선택됩니다:
  * 1은 관찰된 값
  * 0은 누락된 값(즉, 0으로 대체된 NaN 값).

* `static_categorical_features` (`torch.LongTensor` of shape (`batch_size`, number of static categorical features), optional) — Optional 모델이 임베딩을 학습하고 시계열 값에 추가할 정적 범주형 특징.
정적 범주형 특징은 모든 시간 단계에 대해 동일한 값을 갖는 특징입니다(시간에 따라 정적).

정적 범주형 피처의 일반적인 예는 시계열 ID입니다.

`static_real_features` (`torch.FloatTensor` of shape (`batch_size`, number of static real features), optional) — Optional 모델이 시계열 값에 추가할 정적 범주형 특징.
정적 범주형 특징은 모든 시간 단계에 대해 동일한 값을 갖는 특징입니다(시간에 따라 정적).

정적 범주형 특징의 일반적인 예는 프로모션 정보입니다.

`future_values` (`torch.FloatTensor` of shape (`batch_size`, `prediction_length`) or (`batch_size`, `prediction_length`, `input_size`), optional) — 모델의 레이블 역할을 하는 시계열의 미래 값입니다. `future_values는` 과거 값을 고려하여 출력하는 법을 배우기 위해 Transformer가 학습하는 동안 필요한 것입니다.
여기서 시퀀스 길이는 `prediction_length`와 같습니다.

See the demo notebook and code snippets for details.

Optionally, during training any missing values need to be replaced with zeros and indicated via the `future_observed_mask`.

For multivariate time series, the input_size > 1 dimension is required and corresponds to the number of variates in the time series per time step.

* `future_time_features` (`torch.FloatTensor` of shape (`batch_size`, `prediction_length`, `num_features`)) — Required time features for the prediction window, which the model internally will add to future_values. These could be things like “month of year”, “day of the month”, etc. encoded as vectors (for instance as Fourier features). These could also be so-called “age” features, which basically help the model know “at which point in life” a time-series is. Age features have small values for distant past time steps and increase monotonically the more we approach the current time step. Holiday features are also a good example of time features.
These features serve as the “positional encodings” of the inputs. So contrary to a model like BERT, where the position encodings are learned from scratch internally as parameters of the model, the Time Series Transformer requires to provide additional time features. The Time Series Transformer only learns additional embeddings for `static_categorical_features`.

Additional dynamic real covariances can be concatenated to this tensor, with the caveat that these features must but known at prediction time.

The num_features here is equal to `config.num_time_features`+`config.num_dynamic_real_features`.

* `future_observed_mask` (`torch.BoolTensor` of shape (`batch_size`, `sequence_length`) or (`batch_size`, `sequence_length`, `input_size`), optional) — Boolean mask to indicate which future_values were observed and which were missing. Mask values selected in [0, 1]:

  * 1 for values that are observed,
  * 0 for values that are missing (i.e. NaNs that were replaced by zeros).

This mask is used to filter out missing values for the final loss calculation.

* `attention_mask` (`torch.Tensor` of shape (`batch_size`, `sequence_length`), optional) — Mask to avoid performing attention on certain token indices. Mask values selected in [0, 1]:

  * 1 for tokens that are not masked,
  * 0 for tokens that are masked.

What are attention masks?

* `decoder_attention_mask` (`torch.LongTensor` of shape (`batch_size`, `target_sequence_length`), optional) — Mask to avoid performing attention on certain token indices. By default, a causal mask will be used, to make sure the model can only look at previous inputs in order to predict the future.

* `head_mask` (`torch.Tensor` of shape (`encoder_layers`, `encoder_attention_heads`), optional) — Mask to nullify selected heads of the attention modules in the encoder. Mask values selected in [0, 1]:
  * 1 indicates the head is not masked,
  * 0 indicates the head is masked.

* `decoder_head_mask` (`torch.Tensor` of shape (`decoder_layers`, `decoder_attention_heads`), optional) — Mask to nullify selected heads of the attention modules in the decoder. Mask values selected in [0, 1]:
  * 1 indicates the head is not masked,
  * 0 indicates the head is masked.

* `cross_attn_head_mask` (`torch.Tensor` of shape (`decoder_layers`, `decoder_attention_heads`), optional) — Mask to nullify selected heads of the cross-attention modules. Mask values selected in [0, 1]:
  * 1 indicates the head is not masked,
  * 0 indicates the head is masked.

* `encoder_outputs` (tuple(tuple(`torch.FloatTensor`), optional) — Tuple consists of `last_hidden_state`, `hidden_states` (optional) and attentions (optional) `last_hidden_state` of shape (`batch_size`, `sequence_length`, `hidden_size`) (optional) is a sequence of hidden-states at the output of the last layer of the encoder. Used in the cross-attention of the decoder.

* `past_key_values` (tuple(tuple(`torch.FloatTensor`)), optional, returned when `use_cache=True` is passed or when `config.use_cache=True`) — Tuple of tuple(`torch.FloatTensor`) of length `config.n_layers`, with each tuple having 2 tensors of shape (`batch_size`, `num_heads`, `sequence_length`, `embed_size_per_head`)) and 2 additional tensors of shape (`batch_size`, `num_heads`, `encoder_sequence_length`, `embed_size_per_head`).
Contains pre-computed hidden-states (key and values in the self-attention blocks and in the cross-attention blocks) that can be used (see past_key_values input) to speed up sequential decoding.

If `past_key_values` are used, the user can optionally input only the last `decoder_input_ids` (those that don’t have their past key value states given to this model) of shape (`batch_size`, 1) instead of all decoder_input_ids of shape (`batch_size`, `sequence_length`).

`inputs_embeds` (`torch.FloatTensor` of shape (batch_size, sequence_length, hidden_size), optional) — Optionally, instead of passing input_ids you can choose to directly pass an embedded representation. This is useful if you want more control over how to convert input_ids indices into associated vectors than the model’s internal embedding lookup matrix.

* `use_cache` (bool, optional) — If set to True, past_key_values key value states are returned and can be used to speed up decoding (see past_key_values).

* `output_attentions` (bool, optional) — Whether or not to return the attentions tensors of all attention layers. See attentions under returned tensors for more detail.

* `output_hidden_states` (bool, optional) — Whether or not to return the hidden states of all layers. See hidden_states under returned tensors for more detail.

* `return_dict` (bool, optional) — Whether or not to return a ModelOutput instead of a plain tuple.


### Returns

transformers.modeling.outputs.Seq2SeqModelOutput or tuple(torch.FloatTensor)

* `last_hidden_state` (torch.FloatTensor of shape (batch_size, sequence_length, hidden_size)) — Sequence of hidden-states at the output of the last layer of the decoder of the model.

If past_key_values is used only the last hidden-state of the sequences of shape (batch_size, 1, hidden_size) is output.
past_key_values (tuple(tuple(torch.FloatTensor)), optional, returned when use_cache=True is passed or when config.use_cache=True) — Tuple of tuple(torch.FloatTensor) of length config.n_layers, with each tuple having 2 tensors of shape (batch_size, num_heads, sequence_length, embed_size_per_head)) and 2 additional tensors of shape (batch_size, num_heads, encoder_sequence_length, embed_size_per_head).

Contains pre-computed hidden-states (key and values in the self-attention blocks and in the cross-attention blocks) that can be used (see past_key_values input) to speed up sequential decoding.

* `decoder_hidden_states` (tuple(torch.FloatTensor), optional, returned when output_hidden_states=True is passed or when config.output_hidden_states=True) — Tuple of torch.FloatTensor (one for the output of the embeddings, if the model has an embedding layer, + one for the output of each layer) of shape (batch_size, sequence_length, hidden_size).

Hidden-states of the decoder at the output of each layer plus the optional initial embedding outputs.

* `decoder_attentions` (tuple(torch.FloatTensor), optional, returned when output_attentions=True is passed or when config.output_attentions=True) — Tuple of torch.FloatTensor (one for each layer) of shape (batch_size, num_heads, sequence_length, sequence_length).

Attentions weights of the decoder, after the attention softmax, used to compute the weighted average in the self-attention heads.

* `cross_attentions` (tuple(torch.FloatTensor), optional, returned when output_attentions=True is passed or when config.output_attentions=True) — Tuple of torch.FloatTensor (one for each layer) of shape (batch_size, num_heads, sequence_length, sequence_length).

Attentions weights of the decoder’s cross-attention layer, after the attention softmax, used to compute the weighted average in the cross-attention heads.

* `encoder_last_hidden_state` (torch.FloatTensor of shape (batch_size, sequence_length, hidden_size), optional) — Sequence of hidden-states at the output of the last layer of the encoder of the model.

* `encoder_hidden_states` (tuple(torch.FloatTensor), optional, returned when output_hidden_states=True is passed or when config.output_hidden_states=True) — Tuple of torch.FloatTensor (one for the output of the embeddings, if the model has an embedding layer, + one for the output of each layer) of shape (batch_size, sequence_length, hidden_size).

Hidden-states of the encoder at the output of each layer plus the optional initial embedding outputs.

* `encoder_attentions` (tuple(torch.FloatTensor), optional, returned when output_attentions=True is passed or when config.output_attentions=True) — Tuple of torch.FloatTensor (one for each layer) of shape (batch_size, num_heads, sequence_length, sequence_length).

Attentions weights of the encoder, after the attention softmax, used to compute the weighted average in the self-attention heads.
loc (torch.FloatTensor of shape (batch_size,) or (batch_size, input_size), optional) — Shift values of each time series’ context window which is used to give the model inputs of the same magnitude and then used to shift back to the original magnitude.
scale (torch.FloatTensor of shape (batch_size,) or (batch_size, input_size), optional) — Scaling values of each time series’ context window which is used to give the model inputs of the same magnitude and then used to rescale back to the original magnitude.
static_features (torch.FloatTensor of shape (batch_size, feature size), optional) — Static features of each time series’ in a batch which are copied to the covariates at inference time.

In [5]:
# Sample 

from huggingface_hub import hf_hub_download
import torch
from transformers import TimeSeriesTransformerForPrediction

file = hf_hub_download(
    repo_id="hf-internal-testing/tourism-monthly-batch",filename="train-batch.pt", repo_type="dataset"
)

batch = torch.load(file)

model = TimeSeriesTransformerForPrediction.from_pretrained(
    "huggingface/time-series-transformer-tourism-monthly"
)

outputs = model(
    past_values=batch["past_values"],
    past_time_features=batch["past_time_features"],
    past_observed_mask=batch["past_observed_mask"],
    static_categorical_features=batch["static_categorical_features"],
    static_real_features=batch["static_real_features"],
    future_values=batch["future_values"],
    future_time_features=batch["future_time_features"],
)

loss = outputs.loss
loss.backward()

outputs = model.generate(
    past_values=batch["past_values"],
    past_time_features=batch["past_time_features"],
    past_observed_mask=batch["past_observed_mask"],
    static_categorical_features=batch["static_categorical_features"],
    static_real_features=batch["static_real_features"],
    future_time_features=batch["future_time_features"],
)

mean_prediction = outputs.sequences.mean(dim=1)

# Sample
mean_prediction

/tmp/ipykernel_73765/2534584452.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  batch = torch.load(file)


tensor([[25553.8281, 32870.6406, 32968.9297,  ..., 33791.2969, 37489.9219,
         28650.5059],
        [24609.8496, 35064.0195, 47750.0938,  ..., 25898.7383, 26012.3457,
         23417.2031],
        [ 1107.4222,  1135.8021,  1128.0045,  ...,  2121.8477,  1870.3226,
          1667.2338],
        ...,
        [ 2902.1611,  3778.1709,  3529.3625,  ...,  3357.0894,  3908.6863,
          4102.9067],
        [  782.9967,   607.5106,   328.7884,  ...,   586.4850,  1311.8656,
          1181.1353],
        [  387.5719,   619.5315,   423.6380,  ...,   436.6917,   407.5393,
           528.8020]])

In [6]:
from datasets import load_dataset

dataset = load_dataset("asanobm/jena_climate_2009_2016")

dataset

DatasetDict({
    train: Dataset({
        features: ['p (mbar)', 'T (degC)', 'Tpot (K)', 'Tdew (degC)', 'rh (%)', 'VPmax (mbar)', 'VPact (mbar)', 'VPdef (mbar)', 'sh (g/kg)', 'H2OC (mmol/mol)', 'rho (g/m**3)', 'Wx', 'Wy', 'max Wx', 'max Wy', 'Day sin', 'Day cos', 'Year sin', 'Year cos'],
        num_rows: 49063
    })
    validation: Dataset({
        features: ['p (mbar)', 'T (degC)', 'Tpot (K)', 'Tdew (degC)', 'rh (%)', 'VPmax (mbar)', 'VPact (mbar)', 'VPdef (mbar)', 'sh (g/kg)', 'H2OC (mmol/mol)', 'rho (g/m**3)', 'Wx', 'Wy', 'max Wx', 'max Wy', 'Day sin', 'Day cos', 'Year sin', 'Year cos'],
        num_rows: 14018
    })
    test: Dataset({
        features: ['p (mbar)', 'T (degC)', 'Tpot (K)', 'Tdew (degC)', 'rh (%)', 'VPmax (mbar)', 'VPact (mbar)', 'VPdef (mbar)', 'sh (g/kg)', 'H2OC (mmol/mol)', 'rho (g/m**3)', 'Wx', 'Wy', 'max Wx', 'max Wy', 'Day sin', 'Day cos', 'Year sin', 'Year cos'],
        num_rows: 7010
    })
})

,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),Wx,Wy,max Wx,max Wy,Day sin,Day cos,Year sin,Year cos
0,0.945308,-1.982473,-2.041888,-1.918973,1.117102,-1.302851,-1.477323,-0.790424,-1.480036,-1.482697,2.218524,0.193409,0.221161,0.111140,0.217928,0.366111,1.366069,-0.061052,1.428434
1,0.959770,-2.078372,-2.138166,-2.060964,1.044617,-1.330143,-1.534354,-0.786272,-1.536190,-1.539035,2.325708,0.172987,0.222101,0.109458,0.227798,0.707200,1.224794,-0.060029,1.428424
2,0.986284,-2.070284,-2.132435,-2.045187,1.062738,-1.328843,-1.527225,-0.788348,-1.528703,-1.531992,2.323998,0.207983,0.276266,0.111218,0.324078,1.000100,1.000059,-0.059006,1.428412
3,1.004362,-2.098014,-2.161090,-2.096820,1.008375,-1.336641,-1.546235,-0.782121,-1.547420,-1.553119,2.358913,0.270343,0.195267,0.246907,0.145176,1.224850,0.707179,-0.057983,1.428400
4,1.061006,-2.165028,-2.232152,-2.187178,0.984214,-1.353535,-1.579503,-0.782121,-1.581113,-1.585982,2.446320,0.112264,0.350818,0.048640,0.402053,1.366133,0.366112,-0.056960,1.428388
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49058,0.169167,0.904892,0.886564,1.571997,0.947972,0.776492,1.984916,-0.493604,1.979016,1.977352,-0.882480,-0.166720,-1.508412,-0.155728,-1.363530,-0.955374,1.042725,-0.855026,-1.154613
49059,0.266787,0.879473,0.853325,1.405624,0.621793,0.742703,1.680752,-0.283962,1.672043,1.672192,-0.822662,-0.824901,-0.641039,-0.704391,-0.475387,-0.652949,1.254471,-0.855883,-1.154062
49060,0.217374,0.740824,0.719224,1.425703,1.086900,0.559461,1.716396,-0.605690,1.709479,1.707403,-0.716211,-0.030746,0.645277,0.013960,0.600050,-0.306022,1.380734,-0.856739,-1.153511
49061,0.213759,0.710783,0.689423,1.412795,1.147303,0.521772,1.692633,-0.645127,1.687018,1.683929,-0.689842,0.014300,-0.102616,-0.095117,-0.198426,0.061765,1.412909,-0.857595,-1.152959


In [ ]:
# 모델 Config

from transformers import TimeSeriesTransformerConfig

config = TimeSeriesTransformerConfig(
    prediction_length=None, #  예측 길이 (int) — 디코더의 예측 길이. 다시 말해, 모델의 예측 지평선. 이 값은 일반적으로 데이터 세트에 의해 결정되며 적절하게 설정하는 것이 좋습니다.
    context_length=None, # Context length (int, 선택 사항, 기본값은 prediction_length) — 인코더의 컨텍스트 길이. None인 경우, 컨텍스트 길이는 prediction_length와 동일합니다.
    distribution_output=None, # 분배 출력 (문자열, 선택 사항, 기본값은 "student_t") — 모델의 분포 방출 헤드. "Student_t", "normal" 또는 "negative_binomial"일 수 있습니다.
    loss=None, # 손실 (문자열, 선택 사항, 기본값은 "nll") - distribution_output 헤드에 해당하는 모델의 손실 함수. 파라메트릭 분포의 경우 음의 로그 가능성(nll)입니다 - 현재 유일하게 지원되는 것입니다.
    input_size=None # input_size (int, optional, defaults to 1) — The size of the target variable which by default is 1 for univariate targets. Would be > 1 in case of multivariate targets.
)
